## Extracion de Matchs

In [1]:
# Extractor de Datos La Liga - Configuración Principal
# =====================================================
# Este notebook extrae datos de partidos de La Liga temporada por temporada
# y los guarda organizados en Data_Extracted/Data_La_Liga/

import subprocess
import sys
import os
import json
from pathlib import Path
import uuid
from datetime import datetime
import warnings

print("🚀 CONFIGURACIÓN DEL EXTRACTOR DE LA LIGA")
print("=" * 50)

# Configurar rutas principales
BASE_PATH = Path(r"c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_To_Docker")
DATA_EXTRACTED_PATH = BASE_PATH / "Data_Extracted" / "Data_La_Liga"

# Crear directorios si no existen
DATA_EXTRACTED_PATH.mkdir(parents=True, exist_ok=True)

# Instalar SoccerData si no está disponible
try:
    import soccerdata as sd
    print("✅ SoccerData ya está instalado")
except ImportError:
    print("📦 Instalando SoccerData...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "soccerdata"])
    import soccerdata as sd
    print("✅ SoccerData instalado correctamente")

# Importar librerías necesarias
import pandas as pd
import numpy as np

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print(f"✅ Configuración completada")
print(f"📂 Ruta base: {BASE_PATH}")
print(f"📂 Ruta destino: {DATA_EXTRACTED_PATH}")
print(f"🐼 Pandas versión: {pd.__version__}")
print(f"📊 SoccerData versión: {sd.__version__}")

# Definir configuración de temporadas
TEMPORADAS_CONFIG = {
    "2017": {"season": "2017", "folder": "2017-2018"},
    "2018": {"season": "2018", "folder": "2018-2019"},
    "2019": {"season": "2019", "folder": "2019-2020"},
    "2020": {"season": "2020", "folder": "2020-2021"},
    "2021": {"season": "2021", "folder": "2021-2022"},
    "2022": {"season": "2022", "folder": "2022-2023"},
    "2023": {"season": "2023", "folder": "2023-2024"},
    "2024": {"season": "2024", "folder": "2024-2025"}
}

print(f"\n📅 Temporadas configuradas: {list(TEMPORADAS_CONFIG.keys())}")
print("🔧 Corrección aplicada: Importaciones completas para manejo de errores")

🚀 CONFIGURACIÓN DEL EXTRACTOR DE LA LIGA


[11/17/25 09:54:48] INFO     No custom team name replacements found. You can configure these in       ]8;id=975190;file://C:\Users\gerar\AppData\Roaming\Python\Python310\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=527795;file://C:\Users\gerar\AppData\Roaming\Python\Python310\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\gerar\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=705529;file://C:\Users\gerar\AppData\Roaming\Python\Python310\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=427747;file://C:\Users\gerar\AppData\Roaming\Python\Python310\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\gerar\soccerdata\config\league_dict.json.                                    

✅ SoccerData ya está instalado
✅ Configuración completada
📂 Ruta base: c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_To_Docker
📂 Ruta destino: c:\Users\gerar\OneDrive\Desktop\Proyecto_Graduacion\Proyecto_Fase1_CD\Data_To_Docker\Data_Extracted\Data_La_Liga
🐼 Pandas versión: 2.3.1
📊 SoccerData versión: 1.8.7

📅 Temporadas configuradas: ['2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
🔧 Corrección aplicada: Importaciones completas para manejo de errores


In [2]:
# VERIFICACIÓN Y CONFIGURACIÓN DE SCRAPERS CON EXTRACCIÓN DUAL PARA 2021
# ========================================================================

def verificar_y_configurar_scraper(season_year):
    """
    Verifica y configura el mejor scraper disponible para una temporada específica
    Para 2021: intenta ambos FBref y ESPN
    Para otras temporadas: prioridad normal
    """
    league = "ESP-La Liga"
    
    print(f"🔍 Verificando scrapers para temporada {season_year}...")
    
    # Opción 1: FBref (Preferida para todas las temporadas)
    scraper_fbref = None
    try:
        scraper_fbref = sd.FBref(league, season_year)
        test_data = scraper_fbref.read_schedule()
        if test_data is not None and len(test_data) > 0:
            print(f"✅ FBref disponible y funcional para {season_year}")
            print(f"   📊 Datos de prueba: {len(test_data)} partidos encontrados")
        else:
            scraper_fbref = None
    except Exception as e:
        print(f"⚠️ FBref no disponible para {season_year}: {e}")
        scraper_fbref = None
    
    # Para otras temporadas, usar lógica normal
    if season_year != "2021":
        if scraper_fbref:
            return scraper_fbref, 'FBref'
        
        # Probar FotMob como respaldo
        try:
            scraper_fotmob = sd.FotMob(league, season_year)
            test_data = scraper_fotmob.read_schedule()
            if test_data is not None and len(test_data) > 0:
                print(f"✅ FotMob disponible como respaldo para {season_year}")
                return scraper_fotmob, 'FotMob'
        except Exception as e:
            print(f"⚠️ FotMob no disponible para {season_year}: {e}")
        
        # Probar ESPN como último respaldo
        try:
            scraper_espn = sd.ESPN(league, season_year)
            test_data = scraper_espn.read_schedule()
            if test_data is not None and len(test_data) > 0:
                print(f"✅ ESPN disponible como respaldo para {season_year}")
                return scraper_espn, 'ESPN'
        except Exception as e:
            print(f"⚠️ ESPN no disponible para {season_year}: {e}")
        
        return None, None
    
    # Para 2021: devolver ambos scrapers si están disponibles
    scraper_espn = None
    try:
        scraper_espn = sd.ESPN(league, season_year)
        test_data = scraper_espn.read_schedule()
        if test_data is not None and len(test_data) > 0:
            print(f"✅ ESPN disponible para {season_year}")
            print(f"   📊 Datos de prueba: {len(test_data)} partidos encontrados")
        else:
            scraper_espn = None
    except Exception as e:
        print(f"⚠️ ESPN no disponible para {season_year}: {e}")
        scraper_espn = None
    
    # Devolver información de ambos scrapers para 2021
    scrapers_disponibles = []
    if scraper_fbref:
        scrapers_disponibles.append((scraper_fbref, 'FBref'))
    if scraper_espn:
        scrapers_disponibles.append((scraper_espn, 'ESPN'))
    
    if scrapers_disponibles:
        print(f"🎯 Para 2021: {len(scrapers_disponibles)} scrapers disponibles")
        return scrapers_disponibles, 'DUAL_2021'
    else:
        return None, None

def extraer_temporada_dual_2021(season_year):
    """
    Función especial para extraer datos de 2021 usando múltiples fuentes
    """
    print(f"\n🚀 EXTRACCIÓN DUAL TEMPORADA 2021")
    print("=" * 50)
    
    scrapers_info, mode = verificar_y_configurar_scraper(season_year)
    
    if mode != 'DUAL_2021' or not scrapers_info:
        print(f"❌ No hay scrapers duales disponibles para 2021")
        return False
    
    exitos = []
    
    # Extraer de cada scraper disponible
    for scraper, source_name in scrapers_info:
        try:
            print(f"\n📊 Extrayendo de {source_name}...")
            schedule_data = scraper.read_schedule()
            
            if schedule_data is not None and len(schedule_data) > 0:
                print(f"   ✅ {len(schedule_data)} partidos obtenidos de {source_name}")
                
                # Procesar datos
                partidos = procesar_datos_partido(schedule_data, season_year, source_name)
                
                # Guardar datos
                if guardar_datos_temporada(partidos, season_year, source_name):
                    print(f"   🎉 Datos de {source_name} guardados exitosamente")
                    exitos.append(source_name)
                else:
                    print(f"   ❌ Error guardando datos de {source_name}")
            else:
                print(f"   ⚠️ {source_name} no devolvió datos")
                
        except Exception as e:
            print(f"   ❌ Error extrayendo de {source_name}: {e}")
    
    if exitos:
        print(f"\n🎉 EXTRACCIÓN DUAL 2021 COMPLETADA")
        print(f"   ✅ Fuentes exitosas: {', '.join(exitos)}")
        return True
    else:
        print(f"\n❌ Falló la extracción dual para 2021")
        return False

print("✅ Funciones de extracción dual para 2021 cargadas correctamente")
print("🎯 ESPECIAL: 2021 extraerá de FBref Y ESPN simultáneamente")

✅ Funciones de extracción dual para 2021 cargadas correctamente
🎯 ESPECIAL: 2021 extraerá de FBref Y ESPN simultáneamente


In [3]:
# Funciones de Utilidad para Extracción de Datos
# ===============================================

def procesar_datos_partido(schedule_data, season, source_used):
    """
    Procesa los datos del calendario para convertirlos al formato de base de datos
    Soporta múltiples fuentes: FBref, FotMob, ESPN, etc.
    """
    partidos_procesados = []
    
    if schedule_data is None or len(schedule_data) == 0:
        print(f"⚠️ No hay datos para la temporada {season}")
        return partidos_procesados
    
    print(f"🔄 Procesando {len(schedule_data)} partidos de {source_used} para temporada {season}...")
    
    for index, match in schedule_data.iterrows():
        try:
            # CORREGIR: Generar match_id de forma más robusta
            # El index puede ser una tupla, por eso convertimos todo a string
            if hasattr(index, '__iter__') and not isinstance(index, str):
                # Si index es una tupla o lista, convertirla a string
                index_str = '_'.join(str(i) for i in index)
            else:
                index_str = str(index)
            
            # Usar game_id si está disponible, sino generar uno único
            match_id = match.get('game_id', None)
            if match_id is None or str(match_id) == 'nan' or pd.isna(match_id):
                # Generar ID único basado en equipos y fecha
                home_team = str(match.get('home_team', 'Unknown')).replace(' ', '_')
                away_team = str(match.get('away_team', 'Unknown')).replace(' ', '_')
                date_str = str(match.get('date', f'{season}-01-01')).replace('-', '')
                match_id = f"laliga_{season}_{home_team}_vs_{away_team}_{date_str}_{index_str}"
            else:
                match_id = str(match_id)
                
            season_id = f"{season}-{str(int(season)+1)[-2:]}"  # 2017 -> 2017-18
            
            # Extraer información del partido según la fuente
            if source_used == 'FBref':
                home_team = str(match.get('home_team', 'Unknown'))
                away_team = str(match.get('away_team', 'Unknown'))
                
                # Parsear el score "1–1" -> home: 1, away: 1
                score_str = str(match.get('score', '0–0'))
                if '–' in score_str and score_str != 'nan':
                    try:
                        home_score, away_score = score_str.split('–')
                        home_score = int(home_score.strip())
                        away_score = int(away_score.strip())
                    except:
                        home_score, away_score = 0, 0
                else:
                    home_score, away_score = 0, 0
                    
                date_game = str(match.get('date', f'{season}-01-01'))
                matchday = match.get('week', index)
                # Asegurar que matchday sea un entero
                if pd.isna(matchday) or matchday is None:
                    matchday = 1
                else:
                    try:
                        matchday = int(matchday)
                    except:
                        matchday = 1
                
            elif source_used == 'FotMob':
                # Formato específico para FotMob
                home_team = str(match.get('home_team', match.get('home', 'Unknown')))
                away_team = str(match.get('away_team', match.get('away', 'Unknown')))
                
                # FotMob puede tener diferentes formatos de score
                home_score = match.get('home_score', match.get('score_home', 0))
                away_score = match.get('away_score', match.get('score_away', 0))
                
                home_score = int(home_score) if home_score is not None and not pd.isna(home_score) else 0
                away_score = int(away_score) if away_score is not None and not pd.isna(away_score) else 0
                
                date_game = str(match.get('date', match.get('datetime', f'{season}-01-01')))
                matchday = match.get('matchday', match.get('week', match.get('round', 1)))
                try:
                    matchday = int(matchday) if matchday is not None and not pd.isna(matchday) else 1
                except:
                    matchday = 1
                
            elif source_used == 'ESPN':
                # Formato específico para ESPN
                home_team = str(match.get('home_team', match.get('team_home', 'Unknown')))
                away_team = str(match.get('away_team', match.get('team_away', 'Unknown')))
                
                home_score = match.get('home_score', match.get('goals_home', 0))
                away_score = match.get('away_score', match.get('goals_away', 0))
                
                home_score = int(home_score) if home_score is not None and not pd.isna(home_score) else 0
                away_score = int(away_score) if away_score is not None and not pd.isna(away_score) else 0
                
                date_game = str(match.get('date', match.get('match_date', f'{season}-01-01')))
                matchday = match.get('matchday', match.get('week', match.get('gameweek', 1)))
                try:
                    matchday = int(matchday) if matchday is not None and not pd.isna(matchday) else 1
                except:
                    matchday = 1
                
            else:  # Otros scrapers genéricos
                home_team = str(match.get('home_team', match.get('home', 'Unknown')))
                away_team = str(match.get('away_team', match.get('away', 'Unknown')))
                
                h_score = match.get('home_score', 0)
                a_score = match.get('away_score', 0)
                
                home_score = int(h_score) if h_score is not None and not pd.isna(h_score) else 0
                away_score = int(a_score) if a_score is not None and not pd.isna(a_score) else 0
                
                date_game = str(match.get('date', f'{season}-01-01'))
                matchday = match.get('matchday', match.get('week', 1))
                try:
                    matchday = int(matchday) if matchday is not None and not pd.isna(matchday) else 1
                except:
                    matchday = 1
            
            # Limpiar nombres de equipos
            home_team = home_team.replace('Unknown', f'Team_Home_{index_str}')
            away_team = away_team.replace('Unknown', f'Team_Away_{index_str}')
            
            # Crear el registro del partido
            partido = {
                "season_id": str(season_id),
                "match_id": str(match_id),
                "matchday": int(matchday),
                "home_team": str(home_team),
                "home_team_id": f"team_{hash(home_team) % 100000}",
                "home_team_score": int(home_score),
                "away_team": str(away_team),
                "away_team_id": f"team_{hash(away_team) % 100000}",
                "away_team_score": int(away_score),
                "date_game": str(date_game),
                "source": source_used,  # Agregar fuente de datos
                "index_info": str(index_str)  # Para debug
            }
            
            partidos_procesados.append(partido)
            
        except Exception as e:
            print(f"⚠️ Error procesando partido {index}: {e}")
            # Crear un partido de respaldo para no perder el registro
            try:
                if hasattr(index, '__iter__') and not isinstance(index, str):
                    index_str = '_'.join(str(i) for i in index)
                else:
                    index_str = str(index)
                    
                partido_respaldo = {
                    "season_id": f"{season}-{str(int(season)+1)[-2:]}",
                    "match_id": f"error_partido_{season}_{index_str}",
                    "matchday": 1,
                    "home_team": f"Error_Home_{index_str}",
                    "home_team_id": f"team_error_home_{index_str}",
                    "home_team_score": 0,
                    "away_team": f"Error_Away_{index_str}",
                    "away_team_id": f"team_error_away_{index_str}",
                    "away_team_score": 0,
                    "date_game": f"{season}-01-01",
                    "source": f"{source_used}_ERROR",
                    "error_info": str(e)
                }
                partidos_procesados.append(partido_respaldo)
            except:
                pass  # Si incluso el respaldo falla, continuar
            continue
    
    print(f"✅ Procesados {len(partidos_procesados)} partidos correctamente para temporada {season}")
    return partidos_procesados

def guardar_datos_temporada(partidos, season, source_used):
    """
    Guarda los datos de una temporada en el archivo JSON correspondiente
    """
    if not partidos:
        print(f"❌ No hay partidos para guardar en temporada {season}")
        return False
    
    # Crear ruta de la temporada
    temporada_config = TEMPORADAS_CONFIG[season]
    season_folder = DATA_EXTRACTED_PATH / temporada_config["folder"]
    season_folder.mkdir(parents=True, exist_ok=True)
    
    # Nombre del archivo
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    source_clean = source_used.replace(" ", "_").replace("(", "").replace(")", "")
    filename = f"matches_laliga_{season}_{source_clean}_{timestamp}.json"
    output_path = season_folder / filename
    
    # Contar partidos con errores
    partidos_error = sum(1 for p in partidos if "ERROR" in p.get("source", ""))
    partidos_validos = len(partidos) - partidos_error
    
    # Estructura de datos para la base de datos
    database_export = {
        "metadata": {
            "extraction_date": datetime.now().isoformat(),
            "source": source_used,
            "source_priority": "FBref" if "FBref" in source_used else ("FotMob" if "FotMob" in source_used else ("ESPN" if "ESPN" in source_used else "Other")),
            "total_matches": len(partidos),
            "valid_matches": partidos_validos,
            "error_matches": partidos_error,
            "league": "La Liga (Primera División)",
            "season": f"{season}-{str(int(season)+1)[-2:]}",
            "season_folder": temporada_config["folder"],
            "database_table": "matches_registered",
            "data_quality": "Real" if source_used in ["FBref", "FotMob", "ESPN"] else "Demo",
            "note": "Understat removido por datos de temporada incorrectos"
        },
        "matches": partidos
    }
    
    # Guardar el archivo JSON
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(database_export, f, ensure_ascii=False, indent=2)
        
        print(f"✅ TEMPORADA {season} GUARDADA:")
        print(f"   📁 Ubicación: {output_path}")
        print(f"   📊 Total partidos: {len(partidos)}")
        print(f"   ✅ Partidos válidos: {partidos_validos}")
        if partidos_error > 0:
            print(f"   ⚠️ Partidos con errores: {partidos_error}")
        print(f"   🗂️ Fuente: {source_used}")
        print(f"   🎯 Calidad: {'Real' if source_used in ['FBref', 'FotMob', 'ESPN'] else 'Demo'}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error guardando temporada {season}: {e}")
        return False

def extraer_temporada(season_year):
    """
    Extrae datos de una temporada específica usando el mejor scraper disponible
    Prioridad: FBref > FotMob > ESPN > Datos de demostración
    """
    print(f"\n🚀 EXTRAYENDO TEMPORADA {season_year}")
    print("=" * 50)
    
    # Verificar y configurar el mejor scraper disponible
    scraper, source_used = verificar_y_configurar_scraper(season_year)
    
    if scraper and source_used:
        try:
            print(f"✅ Usando {source_used} para temporada {season_year}")
            
            # Obtener datos del scraper configurado
            schedule_data = scraper.read_schedule()
            
            if schedule_data is not None and len(schedule_data) > 0:
                print(f"📊 Datos obtenidos de {source_used}: {len(schedule_data)} partidos")
                
                # Procesar datos
                partidos = procesar_datos_partido(schedule_data, season_year, source_used)
                
                # Guardar datos
                if guardar_datos_temporada(partidos, season_year, source_used):
                    print(f"🎉 Temporada {season_year} completada exitosamente con {source_used}")
                    return True
                else:
                    print(f"❌ Error guardando temporada {season_year}")
                    
            else:
                print(f"⚠️ {source_used} no devolvió datos para temporada {season_year}")
                
        except Exception as e:
            print(f"❌ Error extrayendo temporada {season_year} con {source_used}: {e}")
    
    # Si llegamos aquí, los scrapers fallaron - usar datos de demostración
    print(f"🔄 Generando datos de demostración para temporada {season_year}...")
    
    # Crear datos de demostración realistas
    equipos_laliga = [
        "Real Madrid", "FC Barcelona", "Atlético Madrid", "Sevilla FC",
        "Real Betis", "Real Sociedad", "Villarreal CF", "Athletic Club",
        "Valencia CF", "Celta de Vigo", "RCD Espanyol", "Getafe CF",
        "CA Osasuna", "Rayo Vallecano", "UD Almería", "Real Valladolid",
        "Girona FC", "Cádiz CF", "RCD Mallorca", "Elche CF"
    ]
    
    demo_matches = []
    for i in range(1, 39):  # 38 jornadas típicas de La Liga
        # Seleccionar equipos de forma pseudo-aleatoria
        home_idx = (i * 3) % len(equipos_laliga)
        away_idx = (i * 7) % len(equipos_laliga)
        if home_idx == away_idx:
            away_idx = (away_idx + 1) % len(equipos_laliga)
            
        partido = {
            "season_id": f"{season_year}-{str(int(season_year)+1)[-2:]}",
            "match_id": f"laliga_demo_{season_year}_{i:02d}",
            "matchday": i,
            "home_team": equipos_laliga[home_idx],
            "home_team_id": f"team_{hash(equipos_laliga[home_idx]) % 100000}",
            "home_team_score": (i * 2) % 4,  # Scores entre 0-3
            "away_team": equipos_laliga[away_idx],
            "away_team_id": f"team_{hash(equipos_laliga[away_idx]) % 100000}",
            "away_team_score": (i * 3) % 4,  # Scores entre 0-3
            "date_game": f"{season_year}-{str((i%12)+1).zfill(2)}-{str((i%28)+1).zfill(2)}",
            "source": "Demo Data"
        }
        demo_matches.append(partido)
    
    success = guardar_datos_temporada(demo_matches, season_year, "Demo Data (Scrapers no disponibles)")
    
    if success:
        print(f"✅ Temporada {season_year} completada con datos de demostración")
    else:
        print(f"❌ Error completando temporada {season_year}")
    
    return success

print("✅ Funciones de utilidad mejoradas cargadas correctamente")
print("🎯 NUEVA PRIORIDAD: FBref → FotMob → ESPN → Demo Data")
print("🔧 Corrección aplicada: Manejo robusto de índices de partidos")
print("🚫 NOTA: Understat removido por datos de temporada incorrectos")

✅ Funciones de utilidad mejoradas cargadas correctamente
🎯 NUEVA PRIORIDAD: FBref → FotMob → ESPN → Demo Data
🔧 Corrección aplicada: Manejo robusto de índices de partidos
🚫 NOTA: Understat removido por datos de temporada incorrectos


In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2017-2018
# ================================

print("🏆 PROCESANDO TEMPORADA 2017-2018")
print("=" * 50)

try:
    extraer_temporada("2017")
    print("\n✅ Temporada 2017-2018 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2017-2018'}")
except Exception as e:
    print(f"❌ Error en temporada 2017-2018: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2018-2019
# ================================

print("🏆 PROCESANDO TEMPORADA 2018-2019")
print("=" * 50)

try:
    extraer_temporada("2018")
    print("\n✅ Temporada 2018-2019 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2018-2019'}")
except Exception as e:
    print(f"❌ Error en temporada 2018-2019: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2019-2020
# ================================

print("🏆 PROCESANDO TEMPORADA 2019-2020")
print("=" * 50)

try:
    extraer_temporada("2019")
    print("\n✅ Temporada 2019-2020 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2019-2020'}")
except Exception as e:
    print(f"❌ Error en temporada 2019-2020: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2020-2021
# ================================

print("🏆 PROCESANDO TEMPORADA 2020-2021")
print("=" * 50)

try:
    extraer_temporada("2020")
    print("\n✅ Temporada 2020-2021 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2020-2021'}")
except Exception as e:
    print(f"❌ Error en temporada 2020-2021: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2021-2022
# ================================

print("🏆 PROCESANDO TEMPORADA 2021-2022")
print("=" * 50)

try:
    extraer_temporada("2021")
    print("\n✅ Temporada 2021-2022 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2021-2022'}")
except Exception as e:
    print(f"❌ Error en temporada 2021-2022: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2022-2023
# ================================

print("🏆 PROCESANDO TEMPORADA 2022-2023")
print("=" * 50)

try:
    extraer_temporada("2022")
    print("\n✅ Temporada 2022-2023 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2022-2023'}")
except Exception as e:
    print(f"❌ Error en temporada 2022-2023: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2023-2024
# ================================

print("🏆 PROCESANDO TEMPORADA 2023-2024")
print("=" * 50)

try:
    extraer_temporada("2023")
    print("\n✅ Temporada 2023-2024 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2023-2024'}")
except Exception as e:
    print(f"❌ Error en temporada 2023-2024: {e}")

print("\n" + "="*50)

In [ ]:
# EXTRACCIÓN MATCHES TEMPORADA 2024-2025
# ================================

print("🏆 PROCESANDO TEMPORADA 2024-2025")
print("=" * 50)

try:
    extraer_temporada("2024")
    print("\n✅ Temporada 2024-2025 completada exitosamente")
    print(f"📁 Archivos guardados en: {DATA_EXTRACTED_PATH / '2024-2025'}")
except Exception as e:
    print(f"❌ Error en temporada 2024-2025: {e}")

print("\n" + "="*50)

## Extracion de Match_Stats


In [ ]:
# === CELDA 12A: IMPORTACIONES Y CONFIGURACIÓN BÁSICA ===
# =======================================================
import requests
import time
import random
import pandas as pd
import json
from pathlib import Path
import soccerdata as sd
from datetime import datetime
import os

print("🔧 Importaciones completadas...")

# Lista de User-Agents realistas
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
]

def get_safe_headers():
    """Genera headers HTTP seguros"""
    return {
        'User-Agent': random.choice(USER_AGENTS),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0'
    }

def configure_requests_session():
    """Configura sesión de requests de forma segura"""
    session = requests.Session()
    session.headers.update(get_safe_headers())
    return session

print("✅ Configuración anti-bloqueo aplicada:")
print("  • 4 User-Agents aleatorios")
print("  • Headers HTTP seguros")
print("  • Función de sesión configurada")

# === ESQUEMA DE MAPEO DE CAMPOS ===
CAMPO_MAPPING = {
    'match_id': 'match_id', 'date': 'date', 'time': 'time',
    'comp': 'competition', 'round': 'round', 'venue': 'venue',
    'result': 'result', 'gf': 'goals_for', 'ga': 'goals_against',
    'opponent': 'opponent', 'xg': 'expected_goals', 'xga': 'expected_goals_against',
    'poss': 'possession', 'player': 'player_name', 'pos': 'position',
    'age': 'age', 'min': 'minutes_played', 'gls': 'goals',
    'ast': 'assists', 'sh': 'shots', 'sot': 'shots_on_target',
    'crdy': 'yellow_cards', 'crdr': 'red_cards'
}

print(f"📋 Esquema de mapeo: {len(CAMPO_MAPPING)} campos configurados")
print("🚀 Celda 12A completada - Configuración básica lista")

In [ ]:
# === CELDA 12B: FUNCIÓN DE EXTRACCIÓN PRINCIPAL ===
# ==================================================

def extraer_datos_fbref_corregido(match_id, season, max_intentos=2):
    """
    Extrae datos de FBref usando la API correcta
    """
    print(f"🔍 Extrayendo datos para partido {match_id}...")
    
    # Mapeo de temporadas
    season_mapping = {
        "2017-2018": "2017-18", "2018-2019": "2018-19", "2019-2020": "2019-20",
        "2020-2021": "2020-21", "2021-2022": "2021-22", "2022-2023": "2022-23",
        "2023-2024": "2023-24", "2024-2025": "2024-25"
    }
    
    fbref_season = season_mapping.get(season, season)
    print(f"  📅 Temporada: {season} → FBref: {fbref_season}")
    
    for intento in range(1, max_intentos + 1):
        try:
            print(f"  📊 Intento {intento}/{max_intentos}")
            
            # Delay controlado
            delay = random.uniform(5, 10) if intento == 1 else random.uniform(30, 60)
            print(f"  ⏱️ Delay: {delay:.1f}s")
            time.sleep(delay)
            
            # Inicializar FBref
            fbref = sd.FBref(leagues="ESP-La Liga", seasons=fbref_season)
            time.sleep(random.uniform(2, 4))
            
            # === OBTENER DATOS ===
            print(f"  📅 Obteniendo calendario...")
            try:
                schedule = fbref.read_schedule()
                print(f"  ✅ Calendario: {len(schedule)} partidos")
            except Exception as e:
                print(f"  ⚠️ Error calendario: {str(e)}")
                schedule = pd.DataFrame()
            
            time.sleep(random.uniform(3, 7))
            
            print(f"  🏃 Obteniendo stats de jugadores...")
            try:
                player_stats = fbref.read_player_match_stats(stat_type='summary', match_id=match_id)
                print(f"  ✅ Stats jugadores: {len(player_stats)} registros")
            except Exception as e:
                print(f"  ⚠️ Error player stats: {str(e)}")
                player_stats = pd.DataFrame()
            
            time.sleep(random.uniform(2, 5))
            
            print(f"  ⚽ Obteniendo stats de equipos...")
            try:
                # Para team stats, obtenemos stats generales de temporada
                team_stats = fbref.read_team_season_stats(stat_type='standard')
                print(f"  ✅ Stats equipos: {len(team_stats)} registros")
            except Exception as e:
                print(f"  ⚠️ Error team stats: {str(e)}")
                team_stats = pd.DataFrame()
            
            # === CREAR ESTRUCTURA DE DATOS ===
            datos_completos = {
                'match_info': {
                    'match_id': match_id,
                    'season': season,
                    'fbref_season': fbref_season,
                    'extraction_datetime': datetime.now().isoformat(),
                    'data_source': 'FBref',
                    'extractor_version': 'Corrected_v1.0'
                },
                'schedule_data': {},
                'player_stats': [],
                'team_stats': [],
                'teams': {'home': None, 'away': None}
            }
            
            # Procesar datos...
            # (Lógica de procesamiento simplificada)
            if not player_stats.empty:
                for idx, player_row in player_stats.iterrows():
                    player_data = {}
                    for col in player_stats.columns:
                        campo_estandar = CAMPO_MAPPING.get(col, col)
                        valor = player_row[col]
                        if pd.notna(valor):
                            player_data[campo_estandar] = valor
                    datos_completos['player_stats'].append(player_data)
                
                # Extraer equipos
                if hasattr(player_stats.index, 'get_level_values') and 'team' in player_stats.index.names:
                    teams = player_stats.index.get_level_values('team').unique()
                    if len(teams) >= 1:
                        datos_completos['teams']['home'] = teams[0]
                    if len(teams) >= 2:
                        datos_completos['teams']['away'] = teams[1]
            
            print(f"  ✅ Extracción completada exitosamente")
            print(f"    - Jugadores: {len(datos_completos['player_stats'])} registros")
            
            return datos_completos
            
        except Exception as e:
            error_msg = str(e)
            print(f"  ❌ Error en intento {intento}: {error_msg}")
            
            if intento == max_intentos:
                print(f"  ❌ Fallaron todos los intentos para {match_id}")
                return None
            
            delay = random.uniform(45, 90)
            print(f"  ⏱️ Esperando {delay:.1f}s antes del siguiente intento...")
            time.sleep(delay)
    
    return None

print("🚀 Celda 12B completada - Función de extracción lista")

In [7]:
# 🚀 FASE 2: EXTRACCIÓN DE MATCH STATS COMPLETOS
# Función principal para extraer estadísticas detalladas por jugador y agregarlas por equipo

import time
import random
import requests.adapters
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 💻 CONFIGURACIÓN DE DELAYS PARA NO SOBRECARGAR LA PC
# ====================================================
# Configuración "SUAVE" para no trabar tu computadora
DELAY_CONFIG = {
    "inicio_partido": (5, 10),      # 5-10 segundos antes de cada partido
    "entre_requests": (3, 8),        # 3-8 segundos entre requests de API
    "entre_partidos": (10, 20),      # 10-20 segundos entre partidos
    "reintentos": (10, 20)           # 10-20 segundos base para reintentos
}

print("💻 Configuración SUAVE activada - delays aumentados para proteger tu PC")
print(f"   ⏳ Delay inicio: {DELAY_CONFIG['inicio_partido']} segundos")
print(f"   🔄 Delay entre requests: {DELAY_CONFIG['entre_requests']} segundos") 
print(f"   🏆 Delay entre partidos: {DELAY_CONFIG['entre_partidos']} segundos")

def crear_session_con_anti_blocking():
    """Crear sesión HTTP con configuraciones anti-blocking"""
    session = requests.Session()
    
    # Configurar retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=2,
        status_forcelist=[403, 429, 500, 502, 503, 504],
    )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    # Headers anti-blocking
    session.headers.update({
        'User-Agent': random.choice(USER_AGENTS),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    })
    
    return session

def extraer_datos_fbref_corregido(match_id, temporada, max_reintentos=3):
    """
    Extrae TODOS los datos disponibles por jugador y los agrega por equipo
    usando datos reales de FBref con anti-blocking
    """
    
    print(f"🔍 Extrayendo datos completos para partido {match_id} (temporada {temporada})...")
    
    for intento in range(max_reintentos):
        try:
            # Crear nueva sesión con anti-blocking
            session = crear_session_con_anti_blocking()
            
            # Inicializar FBref con sesión personalizada
            fbref_temp = sd.FBref(leagues="ESP-La Liga", seasons=temporada)
            
            # Delay aleatorio antes de empezar (configuración suave para PC)
            delay = random.uniform(*DELAY_CONFIG['inicio_partido'])
            print(f"  ⏳ Esperando {delay:.1f} segundos...")
            time.sleep(delay)
            
            datos_equipos = {}
            stat_types = ["summary", "passing", "defense", "possession", "misc"]
            
            for stat_type in stat_types:
                try:
                    print(f"  📊 Procesando {stat_type}...")
                    
                    # Delay entre requests (configuración suave para PC)
                    time.sleep(random.uniform(*DELAY_CONFIG['entre_requests']))
                    
                    df = fbref_temp.read_player_match_stats(stat_type=stat_type, match_id=match_id)
                    teams = df.index.get_level_values('team').unique()
                    
                    for team in teams:
                        if team not in datos_equipos:
                            datos_equipos[team] = {}
                        
                        team_data = df.xs(team, level='team')
                        
                        # Procesar todas las columnas numéricas
                        for col in team_data.columns:
                            dtype_str = str(team_data[col].dtype)
                            if any(x in dtype_str.lower() for x in ['int', 'float']):
                                try:
                                    total = team_data[col].sum(skipna=True)
                                    if pd.notna(total) and total != 0:
                                        # Crear nombre de campo limpio
                                        if isinstance(col, tuple):
                                            field_name = f"{stat_type}_{col[0]}_{col[1]}" if col[1] else f"{stat_type}_{col[0]}"
                                        else:
                                            field_name = f"{stat_type}_{col}"
                                        
                                        # Limpiar el nombre del campo
                                        field_name = field_name.replace(' ', '_').replace('%', 'pct')
                                        datos_equipos[team][field_name] = float(total)
                                except:
                                    pass
                except Exception as e:
                    print(f"  ❌ Error con {stat_type}: {e}")
                    continue
            
            if datos_equipos:
                print(f"  ✅ Datos extraídos exitosamente para {len(datos_equipos)} equipos")
                return datos_equipos
            else:
                raise Exception("No se pudieron extraer datos")
                
        except Exception as e:
            print(f"  ⚠️ Intento {intento + 1} falló: {e}")
            if intento < max_reintentos - 1:
                delay = random.uniform(*DELAY_CONFIG['reintentos']) * (intento + 1)
                print(f"  ⏳ Esperando {delay:.1f} segundos antes del siguiente intento...")
                time.sleep(delay)
            else:
                print(f"  ❌ Falló después de {max_reintentos} intentos")
                return None
    
    return None

💻 Configuración SUAVE activada - delays aumentados para proteger tu PC
   ⏳ Delay inicio: (5, 10) segundos
   🔄 Delay entre requests: (3, 8) segundos
   🏆 Delay entre partidos: (10, 20) segundos


In [8]:
def procesar_archivo_multiples_partidos_corregido(archivo_matches, temporada, limite_partidos=None):
    """Procesa archivo de matches y extrae match stats completos para cada partido"""
    
    print(f"📁 Procesando archivo: {archivo_matches}")
    print(f"🗓️ Temporada: {temporada}")
    
    # Leer archivo de matches
    try:
        with open(archivo_matches, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        matches = data.get('matches', [])
        print(f"📊 Total de partidos encontrados: {len(matches)}")
        
        if limite_partidos:
            matches = matches[:limite_partidos]
            print(f"🎯 Procesando solo los primeros {limite_partidos} partidos")
        
    except Exception as e:
        print(f"❌ Error leyendo archivo: {e}")
        return
    
    # Crear carpeta para match stats
    stats_folder = archivo_matches.parent / "Matches_Stats"
    stats_folder.mkdir(exist_ok=True)
    
    exitosos = 0
    errores = 0
    
    for i, match in enumerate(matches, 1):
        match_id = match.get('match_id')
        if not match_id:
            print(f"⚠️ Partido {i}: Sin match_id, saltando...")
            errores += 1
            continue
        
        print(f"\n🏆 Procesando partido {i}/{len(matches)}: {match_id}")
        
        # Verificar si ya existe el archivo
        stats_file = stats_folder / f"match_{match_id}_COMPLETO_stats.json"
        if stats_file.exists():
            print(f"  ✅ Ya existe: {stats_file.name}")
            exitosos += 1
            continue
        
        # Delay adicional entre partidos para no sobrecargar la PC
        if i > 1:  # No delay para el primer partido
            delay_entre_partidos = random.uniform(*DELAY_CONFIG['entre_partidos'])
            print(f"  💻 Pausa para descansar la PC: {delay_entre_partidos:.1f} segundos...")
            time.sleep(delay_entre_partidos)
        
        # Extraer datos con anti-blocking
        datos_equipos = extraer_datos_fbref_corregido(match_id, temporada)
        
        if datos_equipos:
            # Generar JSON completo usando el esquema
            json_completo = generar_json_completo_con_datos_reales(match_id, datos_equipos, match, temporada)
            
            # Guardar archivo
            try:
                with open(stats_file, 'w', encoding='utf-8') as f:
                    json.dump(json_completo, f, indent=2, ensure_ascii=False)
                
                print(f"  ✅ Guardado: {stats_file.name}")
                exitosos += 1
                
            except Exception as e:
                print(f"  ❌ Error guardando: {e}")
                errores += 1
        else:
            print(f"  ❌ No se pudieron extraer datos")
            errores += 1
        
        # Progress report cada 10 partidos
        if i % 10 == 0:
            print(f"\n📊 Progreso: {exitosos} exitosos, {errores} errores de {i} procesados")
    
    print(f"\n🎯 RESUMEN FINAL:")
    print(f"  ✅ Exitosos: {exitosos}")
    print(f"  ❌ Errores: {errores}")
    print(f"  📊 Total procesados: {exitosos + errores}")
    print(f"  📁 Archivos guardados en: {stats_folder}")

def generar_json_completo_con_datos_reales(match_id, datos_equipos, match_info, temporada):
    """
    Genera JSON usando el schema completo con datos reales extraídos de FBref
    """
    
    # Template del JSON resultado
    json_completo = {
        "match_id": match_id,
        "timestamp_extraccion": pd.Timestamp.now().isoformat(),
        "fuente": "SoccerData-FBref-PlayerStats-Completo",
        "temporada": temporada,
        "match_info": match_info,
        "teams": []
    }
    
    # Mapeo de campos de FBref a schema
    mapeo_campos = {
        # Campos básicos
        "summary_Performance_Gls": "ttl_gls",
        "summary_Performance_Ast": "ttl_ast", 
        "summary_Expected_xG": "ttl_xg",
        "summary_Expected_xAG": "ttl_xag",
        "summary_Performance_PK": "ttl_pk_made",
        "summary_Performance_PKatt": "ttl_pk_att",
        "summary_Performance_CrdY": "ttl_yellow_cards",
        "summary_Performance_CrdR": "ttl_red_cards",
        "summary_Performance_Sh": "ttl_sh",
        "summary_Performance_SoT": "ttl_sot",
        
        # Campos de pases
        "passing_Total_Cmp": "ttl_pass_cmp",
        "passing_Total_Att": "ttl_pass_att",
        "passing_PrgP": "ttl_pass_prog",
        "passing_KP": "ttl_key_passes",
        "passing_CrsPA": "ttl_cross_opp_box",
        
        # Campos defensivos
        "defense_Tackles_Tkl": "ttl_tkl",
        "defense_Tackles_TklW": "ttl_tkl_won", 
        "defense_Int": "ttl_int",
        "defense_Blocks_Blocks": "ttl_blocks",
        "defense_Clr": "ttl_clearances",
        "defense_Err": "ttl_def_error",
        
        # Campos de posesión
        "possession_Touches_Touches": "ttl_touches",
        "possession_Touches_Def_Pen": "ttl_touches_def_pen_area",
        "possession_Touches_Att_Pen": "ttl_touches_att_pen_area",
        "possession_Take-Ons_Att": "ttl_take_on_att",
        "possession_Take-Ons_Succ": "ttl_take_on_suc",
        "possession_Carries_Carries": "ttl_carries",
        "possession_Carries_PrgC": "ttl_carries_prog",
        "possession_Carries_Mis": "ttl_carries_miscontrolled",
        "possession_Carries_Dis": "ttl_carries_dispossessed",
        "possession_Receiving_Rec": "ttl_pass_rcvd",
        "possession_Receiving_PrgR": "ttl_pass_prog_rcvd",
        
        # Campos misc
        "misc_Performance_Fls": "ttl_fls_for",
        "misc_Performance_Fld": "ttl_fls_ag",
        "misc_Performance_Off": "ttl_offside",
        "misc_Performance_PKwon": "ttl_pk_won",
        "misc_Performance_PKcon": "ttl_pk_conceded",
        "misc_Performance_OG": "ttl_og",
        "misc_Performance_Recov": "ttl_ball_recov",
        "misc_Aerial_Duels_Won": "ttl_air_dual_won",
        "misc_Aerial_Duels_Lost": "ttl_air_dual_lost"
    }
    
    # Procesar cada equipo
    for i, (equipo_nombre, datos_fbref) in enumerate(datos_equipos.items()):
        equipo_data = {
            "match_id": match_id,
            "season_id": f"La Liga {temporada}",
            "team_id": f"team_{hash(equipo_nombre) % 100000000}",  # ID simple basado en hash
            "team_name": equipo_nombre,
            "home_away": "home" if i == 0 else "away"  # Primer equipo = home
        }
        
        # Mapear campos de FBref a schema
        for campo_fbref, campo_schema in mapeo_campos.items():
            if campo_fbref in datos_fbref:
                equipo_data[campo_schema] = datos_fbref[campo_fbref]
            else:
                equipo_data[campo_schema] = "NA"
        
        # Calcular campos derivados
        if "ttl_sot" in equipo_data and "ttl_sh" in equipo_data and equipo_data["ttl_sh"] != "NA":
            try:
                equipo_data["pct_sot"] = round((equipo_data["ttl_sot"] / equipo_data["ttl_sh"]) * 100, 1) if equipo_data["ttl_sh"] > 0 else 0
            except:
                equipo_data["pct_sot"] = "NA"
        else:
            equipo_data["pct_sot"] = "NA"
            
        if "ttl_pass_cmp" in equipo_data and "ttl_pass_att" in equipo_data and equipo_data["ttl_pass_att"] != "NA":
            try:
                equipo_data["pct_pass_cmp"] = round((equipo_data["ttl_pass_cmp"] / equipo_data["ttl_pass_att"]) * 100, 1) if equipo_data["ttl_pass_att"] > 0 else 0
            except:
                equipo_data["pct_pass_cmp"] = "NA"
        else:
            equipo_data["pct_pass_cmp"] = "NA"
        
        # Campos adicionales con valores por defecto
        campos_adicionales = {
            "ttl_avg_sh": equipo_data.get("ttl_sh", "NA"),
            "players_count": 16,  # Estimado
            "avg_poss": "NA",
            "clean_sheets": "NA",
            "ttl_saves": "NA"
        }
        
        for campo, valor in campos_adicionales.items():
            if campo not in equipo_data:
                equipo_data[campo] = valor
        
        json_completo["teams"].append(equipo_data)
    
    return json_completo

print("✅ Funciones de match stats cargadas correctamente")

✅ Funciones de match stats cargadas correctamente


### 🏆 Temporada 2017-2018 - Extracción Match Stats

In [ ]:
# 🏆 TEMPORADA 2017-2018 - EXTRACCIÓN MATCH STATS
print("="*60)
print("🏆 INICIANDO EXTRACCIÓN DE MATCH STATS - TEMPORADA 2017-2018")
print("="*60)

temporada = "2017-18"
# Usar la ruta correcta encontrada
season_folder = BASE_PATH / "Data_Extracted" / "Data_La_Liga" / "2017-2018"
matches_folder = season_folder / "Matches"

print(f"📁 Buscando archivos en: {matches_folder}")

# Buscar archivos de matches con el patrón correcto
matches_files = list(matches_folder.glob("matches_laliga_*.json"))

if not matches_files:
    print("❌ No se encontraron archivos de matches para 2017-2018")
    print("🔍 Archivos disponibles:")
    if matches_folder.exists():
        all_files = list(matches_folder.glob("*.json"))
        for file in all_files:
            print(f"  - {file.name}")
    else:
        print("  📁 La carpeta no existe")
else:
    print(f"✅ Encontrados {len(matches_files)} archivos de matches:")
    for matches_file in matches_files:
        print(f"  - {matches_file.name}")
    
    # Procesar cada archivo de matches
    for matches_file in matches_files:
        print(f"\n🚀 Procesando: {matches_file.name}")
        
        # Limitar a 2 partidos para prueba inicial (cambiar None para procesar todos)
        limite_partidos = 2  # Cambiar a None para procesar todos los partidos
        
        procesar_archivo_multiples_partidos_corregido(matches_file, temporada, limite_partidos)
        
        print(f"✅ Completado: {matches_file.name}")

print(f"\n🎯 TEMPORADA 2017-2018 COMPLETADA")
print("="*60)

### 🏆 Temporada 2018-2019 - Extracción Match Stats

In [ ]:
# 🏆 TEMPORADA 2018-2019 - EXTRACCIÓN MATCH STATS
print("="*60)
print("🏆 INICIANDO EXTRACCIÓN DE MATCH STATS - TEMPORADA 2018-2019")
print("="*60)

temporada = "2018-19"
# Usar la ruta correcta
season_folder = BASE_PATH / "Data_Extracted" / "Data_La_Liga" / "2018-2019"
matches_folder = season_folder / "Matches"

print(f"📁 Buscando archivos en: {matches_folder}")

# Buscar archivos de matches con el patrón correcto
matches_files = list(matches_folder.glob("matches_laliga_*.json"))

if not matches_files:
    print("❌ No se encontraron archivos de matches para 2018-2019")
    print("🔍 Archivos disponibles:")
    if matches_folder.exists():
        all_files = list(matches_folder.glob("*.json"))
        for file in all_files:
            print(f"  - {file.name}")
    else:
        print("  📁 La carpeta no existe")
else:
    print(f"✅ Encontrados {len(matches_files)} archivos de matches:")
    for matches_file in matches_files:
        print(f"  - {matches_file.name}")
    
    # Procesar cada archivo de matches
    for matches_file in matches_files:
        print(f"\n🚀 Procesando: {matches_file.name}")
        
        # Limitar a 2 partidos para prueba (cambiar None para procesar todos)
        limite_partidos = None  # Cambiar a None para procesar todos los partidos
        
        procesar_archivo_multiples_partidos_corregido(matches_file, temporada, limite_partidos)
        
        print(f"✅ Completado: {matches_file.name}")

print(f"\n🎯 TEMPORADA 2018-2019 COMPLETADA")
print("="*60)

In [ ]:
# 🏆 TEMPORADA 2019-2020 - EXTRACCIÓN MATCH STATS
print("="*60)
print("🏆 INICIANDO EXTRACCIÓN DE MATCH STATS - TEMPORADA 2019-2020")
print("="*60)

temporada = "2019-20"
# Usar la ruta correcta
season_folder = BASE_PATH / "Data_Extracted" / "Data_La_Liga" / "2019-2020"
matches_folder = season_folder / "Matches"

print(f"📁 Buscando archivos en: {matches_folder}")

# Buscar archivos de matches con el patrón correcto
matches_files = list(matches_folder.glob("matches_laliga_*.json"))

if not matches_files:
    print("❌ No se encontraron archivos de matches para 2019-2020")
    print("🔍 Archivos disponibles:")
    if matches_folder.exists():
        all_files = list(matches_folder.glob("*.json"))
        for file in all_files:
            print(f"  - {file.name}")
    else:
        print("  📁 La carpeta no existe")
else:
    print(f"✅ Encontrados {len(matches_files)} archivos de matches:")
    for matches_file in matches_files:
        print(f"  - {matches_file.name}")
    
    # Procesar cada archivo de matches
    for matches_file in matches_files:
        print(f"\n🚀 Procesando: {matches_file.name}")
        
        # Limitar a 2 partidos para prueba (cambiar None para procesar todos)
        limite_partidos = None  # Cambiar a None para procesar todos los partidos
        
        procesar_archivo_multiples_partidos_corregido(matches_file, temporada, limite_partidos)
        
        print(f"✅ Completado: {matches_file.name}")

print(f"\n🎯 TEMPORADA 2019-2020 COMPLETADA")
print("="*60)

In [ ]:
# 🏆 TEMPORADA 2020-2021 - EXTRACCIÓN MATCH STATS
print("="*60)
print("🏆 INICIANDO EXTRACCIÓN DE MATCH STATS - TEMPORADA 2020-2021")
print("="*60)

temporada = "2020-21"
# Usar la ruta correcta
season_folder = BASE_PATH / "Data_Extracted" / "Data_La_Liga" / "2020-2021"
matches_folder = season_folder / "Matches"

print(f"📁 Buscando archivos en: {matches_folder}")

# Buscar archivos de matches con el patrón correcto
matches_files = list(matches_folder.glob("matches_laliga_*.json"))

if not matches_files:
    print("❌ No se encontraron archivos de matches para 2020-2021")
    print("🔍 Archivos disponibles:")
    if matches_folder.exists():
        all_files = list(matches_folder.glob("*.json"))
        for file in all_files:
            print(f"  - {file.name}")
    else:
        print("  📁 La carpeta no existe")
else:
    print(f"✅ Encontrados {len(matches_files)} archivos de matches:")
    for matches_file in matches_files:
        print(f"  - {matches_file.name}")
    
    # Procesar cada archivo de matches
    for matches_file in matches_files:
        print(f"\n🚀 Procesando: {matches_file.name}")
        
        # Limitar a 2 partidos para prueba (cambiar None para procesar todos)
        limite_partidos =  None  # Cambiar a None para procesar todos los partidos
        
        procesar_archivo_multiples_partidos_corregido(matches_file, temporada, limite_partidos)
        
        print(f"✅ Completado: {matches_file.name}")

print(f"\n🎯 TEMPORADA 2020-2021 COMPLETADA")
print("="*60)

In [ ]:
# 🏆 TEMPORADA 2022-2023 - EXTRACCIÓN MATCH STATS
print("="*60)
print("🏆 INICIANDO EXTRACCIÓN DE MATCH STATS - TEMPORADA 2022-2023")
print("="*60)

temporada = "2022-23"
# Usar la ruta correcta
season_folder = BASE_PATH / "Data_Extracted" / "Data_La_Liga" / "2022-2023"
matches_folder = season_folder / "Matches"

print(f"📁 Buscando archivos en: {matches_folder}")

# Buscar archivos de matches con el patrón correcto
matches_files = list(matches_folder.glob("matches_laliga_*.json"))

if not matches_files:
    print("❌ No se encontraron archivos de matches para 2022-2023")
    print("🔍 Archivos disponibles:")
    if matches_folder.exists():
        all_files = list(matches_folder.glob("*.json"))
        for file in all_files:
            print(f"  - {file.name}")
    else:
        print("  📁 La carpeta no existe")
else:
    print(f"✅ Encontrados {len(matches_files)} archivos de matches:")
    for matches_file in matches_files:
        print(f"  - {matches_file.name}")
    
    # Procesar cada archivo de matches
    for matches_file in matches_files:
        print(f"\n🚀 Procesando: {matches_file.name}")
        
        # Limitar a 2 partidos para prueba (cambiar None para procesar todos)
        limite_partidos =  None  # Cambiar a None para procesar todos los partidos
        
        procesar_archivo_multiples_partidos_corregido(matches_file, temporada, limite_partidos)
        
        print(f"✅ Completado: {matches_file.name}")

print(f"\n🎯 TEMPORADA 2022-2023 COMPLETADA")
print("="*60)

In [ ]:
# 🏆 PROCESAMIENTO MASIVO DE TODAS LAS TEMPORADAS RESTANTES
# Esta celda procesa todas las temporadas desde 2019-2020 hasta 2024-2025

temporadas_restantes = [
    ("2019-2020", "2019-20"),
    ("2020-2021", "2020-21"), 
    ("2021-2022", "2021-22"),
    ("2022-2023", "2022-23"),
    ("2023-2024", "2023-24"),
    ("2024-2025", "2024-25")
]

print("🚀 PROCESAMIENTO MASIVO DE MATCH STATS")
print("="*60)

for season_folder_name, temporada_fbref in temporadas_restantes:
    print(f"\n🏆 PROCESANDO TEMPORADA: {season_folder_name} (FBref: {temporada_fbref})")
    print("-" * 50)
    
    # Usar la ruta correcta
    season_folder = BASE_PATH / "Data_Extracted" / "Data_La_Liga" / season_folder_name
    matches_folder = season_folder / "Matches"
    
    print(f"📁 Buscando archivos en: {matches_folder}")
    
    # Buscar archivos de matches con el patrón correcto
    matches_files = list(matches_folder.glob("matches_laliga_*.json"))
    
    if not matches_files:
        print(f"❌ No se encontraron archivos de matches para {season_folder_name}")
        print("🔍 Archivos disponibles:")
        if matches_folder.exists():
            all_files = list(matches_folder.glob("*.json"))
            for file in all_files:
                print(f"  - {file.name}")
        else:
            print("  📁 La carpeta no existe")
        continue
    
    print(f"✅ Encontrados {len(matches_files)} archivos de matches:")
    for matches_file in matches_files:
        print(f"  - {matches_file.name}")
    
    # Procesar cada archivo de matches
    for matches_file in matches_files:
        print(f"\n🚀 Procesando: {matches_file.name}")
        
        # Limitar a 2 partidos para prueba de cada temporada (cambiar None para procesar todos)
        limite_partidos = 2  # Cambiar a None para procesar todos los partidos
        
        try:
            procesar_archivo_multiples_partidos_corregido(matches_file, temporada_fbref, limite_partidos)
            print(f"✅ Completado: {matches_file.name}")
        except Exception as e:
            print(f"❌ Error procesando {matches_file.name}: {e}")
    
    print(f"🎯 TEMPORADA {season_folder_name} COMPLETADA")

print("\n" + "="*60)
print("🎉 PROCESAMIENTO MASIVO COMPLETADO")
print("="*60)

In [6]:
# === CELDA 12C: FUNCIÓN DE PROCESAMIENTO DE ARCHIVOS ===
# =======================================================

def procesar_archivo_multiples_partidos_corregido(archivo_path, season_name, limite_partidos=None):
    """
    Procesa archivo con múltiples partidos usando la API corregida
    """
    print(f"\n🏆 Procesando archivo: {archivo_path.name}")
    print(f"🔧 Usando API corregida de soccerdata")
    
    # Leer archivo
    with open(archivo_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if 'matches' not in data:
        print(f"❌ Archivo no tiene estructura 'matches'")
        return
        
    matches = data['matches']
    total_matches = len(matches)
    
    if limite_partidos:
        matches = matches[:limite_partidos]
        print(f"🔢 Limitando a {limite_partidos} partidos de {total_matches} totales")
    
    print(f"📊 Total de partidos a procesar: {len(matches)}")
    
    # Crear directorios
    matches_output_dir = DATA_EXTRACTED_PATH / season_name / "Matches"
    stats_output_dir = DATA_EXTRACTED_PATH / season_name / "Matches_Stats"
    
    matches_output_dir.mkdir(parents=True, exist_ok=True)
    stats_output_dir.mkdir(parents=True, exist_ok=True)
    
    # Contadores
    exitosos = 0
    errores = 0
    saltados = 0
    
    # Procesar cada partido
    for i, match_data in enumerate(matches, 1):
        try:
            match_id = match_data.get('match_id')
            if not match_id:
                print(f"❌ Sin match_id en partido {i}")
                errores += 1
                continue
            
            home_team = match_data.get('home_team', 'Unknown')
            away_team = match_data.get('away_team', 'Unknown')
            
            print(f"\n📍 Partido {i}/{len(matches)}: {home_team} vs {away_team} (ID: {match_id})")
            
            # Verificar si ya existe
            output_file = matches_output_dir / f"match_{match_id}_COMPLETO.json"
            if output_file.exists():
                print(f"  ⏭️ Ya existe, saltando...")
                saltados += 1
                continue
            
            # Extraer datos usando API corregida
            datos_fbref = extraer_datos_fbref_corregido(match_id, season_name)
            
            if datos_fbref is None:
                print(f"  ❌ Error en extracción")
                errores += 1
                continue
            
            # Combinar datos
            datos_combinados = {
                **match_data,
                'fbref_data': datos_fbref,
                'processing_info': {
                    'processed_at': datetime.now().isoformat(),
                    'source_file': archivo_path.name,
                    'fbref_season': datos_fbref['match_info']['fbref_season'],
                    'extractor_version': 'Corrected_v1.0'
                }
            }
            
            # Guardar archivos
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(datos_combinados, f, ensure_ascii=False, indent=2)
            
            stats_file = stats_output_dir / f"match_{match_id}_player_stats.json"
            stats_data = {
                'match_id': match_id,
                'match_info': datos_fbref['match_info'],
                'teams': datos_fbref['teams'],
                'player_stats': datos_fbref['player_stats'],
                'team_stats': datos_fbref['team_stats']
            }
            
            with open(stats_file, 'w', encoding='utf-8') as f:
                json.dump(stats_data, f, ensure_ascii=False, indent=2)
            
            print(f"  ✅ Procesado exitosamente")
            exitosos += 1
            
            # Delay entre partidos
            if i < len(matches):
                delay = random.uniform(20, 40)
                print(f"  ⏱️ Pausa entre partidos: {delay:.1f}s")
                time.sleep(delay)
        
        except Exception as e:
            print(f"  ❌ Error inesperado: {str(e)}")
            errores += 1
            continue
    
    # Resumen final
    print(f"\n📋 Resumen de procesamiento:")
    print(f"  ✅ Exitosos: {exitosos}")
    print(f"  ⏭️ Saltados: {saltados}")
    print(f"  ❌ Errores: {errores}")
    print(f"  📊 Total: {len(matches)}")
    if len(matches) > saltados:
        success_rate = exitosos/(len(matches)-saltados)*100
        print(f"  🎯 Tasa de éxito: {success_rate:.1f}%")

print("🚀 Celda 12C completada - Función de procesamiento lista")
print("\n🎉 TODAS LAS FUNCIONES DIVIDIDAS Y LISTAS:")
print("  • Celda 12A: Configuración básica ✅")
print("  • Celda 12B: Función de extracción ✅") 
print("  • Celda 12C: Función de procesamiento ✅")
print("🔧 API corregida: read_schedule, read_player_match_stats, read_team_match_stats")

🚀 Celda 12C completada - Función de procesamiento lista

🎉 TODAS LAS FUNCIONES DIVIDIDAS Y LISTAS:
  • Celda 12A: Configuración básica ✅
  • Celda 12B: Función de extracción ✅
  • Celda 12C: Función de procesamiento ✅
🔧 API corregida: read_schedule, read_player_match_stats, read_team_match_stats


In [13]:
# VERIFICAR API DE SOCCERDATA
# ============================

print("🔍 Verificando métodos disponibles en soccerdata...")

# Crear instancia de FBref
fbref = sd.FBref(leagues="ESP-La Liga", seasons="2023-24")

# Verificar métodos disponibles
metodos_disponibles = [metodo for metodo in dir(fbref) if not metodo.startswith('_')]
print(f"📋 Métodos disponibles en FBref:")
for metodo in sorted(metodos_disponibles):
    if 'read' in metodo.lower() or 'match' in metodo.lower():
        print(f"  • {metodo}")

print(f"\n🔧 Total de métodos públicos: {len(metodos_disponibles)}")

# Intentar algunos métodos comunes
metodos_prueba = ['read_games', 'read_schedules', 'read_player_season_stats', 'read_team_season_stats']

for metodo in metodos_prueba:
    if hasattr(fbref, metodo):
        print(f"✅ {metodo} - DISPONIBLE")
    else:
        print(f"❌ {metodo} - NO DISPONIBLE")

🔍 Verificando métodos disponibles en soccerdata...


[11/13/25 22:53:44] INFO     Saving cached data to C:\Users\gerar\soccerdata\data\FBref              ]8;id=669049;file://C:\Users\gerar\AppData\Roaming\Python\Python310\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=765651;file://C:\Users\gerar\AppData\Roaming\Python\Python310\site-packages\soccerdata\_common.py#263\263]8;;\

📋 Métodos disponibles en FBref:
  • read_events
  • read_leagues
  • read_lineup
  • read_player_match_stats
  • read_player_season_stats
  • read_schedule
  • read_seasons
  • read_shot_events
  • read_team_match_stats
  • read_team_season_stats

🔧 Total de métodos públicos: 20
❌ read_games - NO DISPONIBLE
❌ read_schedules - NO DISPONIBLE
✅ read_player_season_stats - DISPONIBLE
✅ read_team_season_stats - DISPONIBLE
